# Sprawozdanie wykonane przez:
# Maksyma Plakushko

 # LAB 2 Metody analizy i filtracji danych w domenie częstotliwościowej



In [ ]:
# Встановлення бібліотек (якщо не встановлені)
# !pip install librosa scipy matplotlib

import os
import random
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.signal import find_peaks, butter, lfilter, freqz, windows
from google.colab import drive

# 1. Монтування Google Drive
drive.mount('/content/drive')

# 2. Вказати шлях до папки audio на Google Drive
# Якщо у вас папка UrbanSound8K знаходиться на головній сторінці Drive,
# то шлях буде виглядати приблизно так:
base_path = '/content/drive/MyDrive/urbansound8k/UrbanSound8K/audio/'

# Перевірка, чи існує вказаний шлях
if not os.path.exists(base_path):
    print(f"Помилка: Шлях {base_path} не знайдено на Google Drive.")
    print("Перевірте, чи правильно вказано шлях до папки audio в UrbanSound8K.")
else:
    # Отримуємо список всіх .wav файлів
    audio_files = []
    # os.walk пройде по всіх підпапках в base_path
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.wav'):
                audio_files.append(os.path.join(root, file))

    if not audio_files:
        print(f"Не знайдено .wav аудіофайлів в директорії {base_path} та її підпапках.")
        print("Перевірте структуру папок UrbanSound8K на Google Drive.")
    else:
        # Вибираємо випадковий файл
        audio_path = random.choice(audio_files)
        print(f"Обраний аудіофайл: {audio_path}")

        # Завантажуємо аудіофайл
        y, sr = librosa.load(audio_path, sr=None)

        # Показуємо хвилю
        plt.figure(figsize=(12, 2))
        librosa.display.waveshow(y, sr=sr)
        plt.title(f'Аудіосигнал: {os.path.basename(audio_path)}')
        plt.show()

        # 2. Аналіз спектра потужності з різними вікнами
        window_types = {
            'rectangular': np.ones(2048),
            'hann': windows.hann(2048),
            'hamming': windows.hamming(2048),
            'blackmanharris': windows.blackmanharris(2048)
        }

        plt.figure(figsize=(16, 8))
        for i, (name, window) in enumerate(window_types.items()):
            # Переконайтеся, що довжина y_win відповідає довжині вікна
            min_len = min(len(y), len(window))
            y_win = y[:min_len] * window[:min_len]

            spectrum = np.abs(np.fft.rfft(y_win))**2
            freqs = np.fft.rfftfreq(len(y_win), 1/sr)
            plt.subplot(2, 2, i+1)
            plt.plot(freqs, spectrum)
            plt.title(f'Power Spectrum ({name} window)')
            plt.xlabel('Frequency [Hz]')
            plt.ylabel('Power')
        plt.tight_layout()
        plt.show()

        # 3. Пошук піків у спектрі потужності з різними параметрами
        peak_params = [
            {'height': 500},
            {'height': 1000},
            {'height': 1500},
            {'threshold': 50},
            {'threshold': 100},
            {'distance': 10},
            {'distance': 20}
        ]

        for name, window in window_types.items():
            print(f"\n{name.upper()} WINDOW")
            min_len = min(len(y), len(window))
            y_win = y[:min_len] * window[:min_len]
            spectrum = np.abs(np.fft.rfft(y_win))**2
            freqs = np.fft.rfftfreq(len(y_win), 1/sr)
            plt.figure(figsize=(12, 6))
            plt.plot(freqs, spectrum, label='Power Spectrum')
            for param in peak_params:
                peaks, props = find_peaks(spectrum, **param)
                plt.plot(freqs[peaks], spectrum[peaks], 'x', label=str(param))
                print(f"Param {param}: Number of peaks = {len(peaks)}")
            plt.title(f'Peaks in Power Spectrum ({name} window)')
            plt.xlabel('Frequency [Hz]')
            plt.ylabel('Power')
            plt.legend()
            plt.show()

        # 4. Спектрограма з різними довжинами вікна та hop_length
        window_lengths = [256, 512, 1024, 2048]
        hop_lengths = [64, 128, 256]

        for n_fft in window_lengths:
            for hop in hop_lengths:
                # Перевірка, чи довжина сигналу достатня для STFT з поточними параметрами
                if len(y) >= n_fft:
                    D = librosa.stft(y, n_fft=n_fft, hop_length=hop)
                    S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
                    plt.figure(figsize=(10, 4))
                    librosa.display.specshow(S_db, sr=sr, hop_length=hop, x_axis='time', y_axis='hz')
                    plt.colorbar(format='%+2.0f dB')
                    plt.title(f'Spectrogram (n_fft={n_fft}, hop_length={hop})')
                    plt.tight_layout()
                    plt.show()
                else:
                    print(f"Сигнал занадто короткий для n_fft={n_fft}. Пропускаємо.")

        # 5. Фільтрація сигналу: проектування та застосування фільтрів
        def butter_filter(data, cutoff, sr, order=5, btype='low'):
            nyq = 0.5 * sr
            normal_cutoff = cutoff / nyq
            # Перевірка на правильні параметри фільтра
            if normal_cutoff >= 1 or normal_cutoff <= 0:
                 print(f"Помилка: Неправильний нормалізований зріз частоти {normal_cutoff}. Пропускаємо фільтрацію.")
                 return data # Повертаємо оригінальні дані або обробляємо помилку

            b, a = butter(order, normal_cutoff, btype=btype, analog=False)
            y_filt = lfilter(b, a, data)
            return y_filt

        low_cutoffs = [500, 1000, 2000]
        high_cutoffs = [1000, 2000, 4000]
        order = 4

        for cutoff in low_cutoffs:
            print(f"Застосування Low-pass фільтра з cutoff={cutoff} Hz")
            y_low = butter_filter(y, cutoff, sr, order=order, btype='low')
            if y_low is not y: # Перевіряємо, чи фільтрація була застосована успішно
                plt.figure(figsize=(12, 2))
                librosa.display.waveshow(y_low, sr=sr)
                plt.title(f'Low-pass Filtered (cutoff={cutoff} Hz)')
                plt.show()
                # Спектрограма після фільтрації
                D = librosa.stft(y_low, n_fft=1024)
                S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
                plt.figure(figsize=(10, 4))
                librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='hz')
                plt.title(f'Spectrogram after Low-pass Filter (cutoff={cutoff} Hz)')
                plt.colorbar(format='%+2.0f dB')
                plt.tight_layout()
                plt.show()

        for cutoff in high_cutoffs:
            print(f"Застосування High-pass фільтра з cutoff={cutoff} Hz")
            y_high = butter_filter(y, cutoff, sr, order=order, btype='high')
            if y_high is not y: # Перевіряємо, чи фільтрація була застосована успішно
                plt.figure(figsize=(12, 2))
                librosa.display.waveshow(y_high, sr=sr)
                plt.title(f'High-pass Filtered (cutoff={cutoff} Hz)')
                plt.show()
                # Спектрограма після фільтрації
                D = librosa.stft(y_high, n_fft=1024)
                S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
                plt.figure(figsize=(10, 4))
                librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='hz')
                plt.title(f'Spectrogram after High-pass Filter (cutoff={cutoff} Hz)')
                plt.colorbar(format='%+2.0f dB')
                plt.tight_layout()
                plt.show()

IndexError: Cannot choose from an empty sequence